# Laboratorio 8: Random Forest y despliegues

**Duración:** 2 horas  
**Formato:** Implementación, despliegue y competencia  

---

## Portada del equipo

**Integrantes:**
- Alanís González Sebástian
- Arano Bejarano Melisa Asharet 
- Fonseca González Bruno 
- Morales Flores Luis Enrique

**Repositorio del equipo:**  
<https://github.com/lukemorales13/random-forest-api>

**Fecha de entrega:**  
31/Octubre/2025

## Elemento 1 - Implementación del Random Forest

In [ ]:
from typing import Optional, Union
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import LabelEncoder


class SimpleRandomForest:
    """
    Random Forest minimalista (bagging de árboles) con interfaz estilo scikit-learn.

    - Usa bootstrap de ejemplos para cada árbol.
    - Usa submuestreo aleatorio de características por división a través de `max_features`
      (delegado al árbol base de scikit-learn).
    - Agregación:
        * Clasificación: promedio de probabilidades y argmax.
        * Regresión: promedio de predicciones.

    Parámetros
    ----------
    n_estimators : int, default=100
        Número de árboles en el ensamble.
    max_features : {'sqrt','log2','all'} o int o float en (0,1], default='sqrt'
        Número de características a considerar en cada división del árbol base.
        Se pasa directo al árbol (traducción: 'all' -> None).
    criterion : str, default='gini' (clasificación) o 'squared_error' (regresión)
        Criterio del árbol. Si task='auto', se ignora este valor y se elige por tipo.
    max_depth : int o None, default=None
        Profundidad máxima de cada árbol.
    random_state : int o None, default=None
        Semilla global para reproducibilidad.
    task : {'auto','classification','regression'}, default='auto'
        Tipo de problema. Si 'auto', se infiere a partir de y.

    Atributos (tras fit)
    --------------------
    estimators_ : list
        Lista de árboles entrenados.
    task_ : str
        Tipo inferido: 'classification' o 'regression'.
    classes_ : np.ndarray (solo clasificación)
        Clases en el orden usado para agregación.
    n_features_in_ : int
        Número de características de entrada.
    """

    def __init__(
        self,
        n_estimators: int = 100,
        max_features: Union[str, int, float] = "sqrt",
        criterion: Optional[str] = None,
        max_depth: Optional[int] = None,
        random_state: Optional[int] = None,
        task: str = "auto",
    ):
        self.n_estimators = int(n_estimators)
        self.max_features = max_features
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.task = task

        # Set en fit
        self.estimators_ = []
        self.task_ = None
        self.classes_ = None
        self._le = None
        self.n_features_in_ = None
        self._rng = None

    # Utilidades internas
    def _resolve_max_features(self, n_features: int):
        """Traduce el argumento max_features a lo que esperan los árboles de sklearn."""
        mf = self.max_features
        if mf == "all":
            return None  # sklearn: None => usa todas
        if isinstance(mf, str):
            if mf in {"sqrt", "log2"}:
                return mf
            raise ValueError("max_features string debe ser 'sqrt', 'log2' o 'all'.")
        if isinstance(mf, (int, np.integer)):
            if 1 <= mf <= n_features:
                return int(mf)
            raise ValueError("max_features int debe estar en [1, n_features].")
        if isinstance(mf, float):
            if 0.0 < mf <= 1.0:
                k = max(1, int(np.floor(mf * n_features)))
                return k
            raise ValueError("max_features float debe estar en (0, 1].")
        raise ValueError("max_features no válido.")
    # Muestra bootstrap
    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray, rng: np.random.RandomState):
        """Devuelve una muestra bootstrap de (X, y) del mismo tamaño que el conjunto original."""
        n_samples = X.shape[0]
        idx = rng.randint(0, n_samples, size=n_samples)  # con reemplazo
        return X[idx], y[idx]
    # Inferencia de tarea
    def _infer_task(self, y: np.ndarray) -> str:
        if self.task in ("classification", "regression"):
            return self.task
        y_type = type_of_target(y)
        if y_type in ("binary", "multiclass"):
            return "classification"
        elif y_type in ("continuous",):
            return "regression"
        else:
            return "classification" if np.issubdtype(np.asarray(y).dtype, np.integer) else "regression"

    # API principal
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X debe ser 2D (n_samples, n_features).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X y y deben tener el mismo número de filas.")

        self.n_features_in_ = X.shape[1]
        self.task_ = self._infer_task(y)
        self._rng = np.random.RandomState(self.random_state)

        max_features = self._resolve_max_features(self.n_features_in_)

        self.estimators_ = []

        if self.task_ == "classification":
            # Aseguramos un orden global de clases para la agregación
            self._le = LabelEncoder().fit(y)
            self.classes_ = self._le.classes_

            # Criterio por defecto si no se especifica
            criterion = self.criterion if self.criterion is not None else "gini"

            for _ in range(self.n_estimators):
                Xi, yi = self._bootstrap_sample(X, y, self._rng)
                seed = int(self._rng.randint(0, 2**31 - 1))
                tree = DecisionTreeClassifier(
                    criterion=criterion,
                    max_depth=self.max_depth,
                    max_features=max_features,
                    random_state=seed,
                )
                tree.fit(Xi, yi)
                self.estimators_.append(tree)

        else:  # regresión
            criterion = self.criterion if self.criterion is not None else "squared_error"

            for _ in range(self.n_estimators):
                Xi, yi = self._bootstrap_sample(X, y, self._rng)
                seed = int(self._rng.randint(0, 2**31 - 1))
                tree = DecisionTreeRegressor(
                    criterion=criterion,
                    max_depth=self.max_depth,
                    max_features=max_features,
                    random_state=seed,
                )
                tree.fit(Xi, yi)
                self.estimators_.append(tree)

        return self
    # Predicción
    def predict(self, X: np.ndarray) -> np.ndarray:
        if not self.estimators_:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")

        X = np.asarray(X)

        if self.task_ == "classification":
            # Promedio de probabilidades con alineación de clases
            proba = self.predict_proba(X)
            # argmax sobre el eje de clases y deshacer codificación
            y_ind = np.argmax(proba, axis=1)
            return self.classes_[y_ind]
        else:
            # Promedio de predicciones de regresión
            preds = np.column_stack([est.predict(X) for est in self.estimators_])
            return preds.mean(axis=1)
    # Probabilidades para clasificación
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """Probabilidades promedio para clasificación. No disponible para regresión."""
        if self.task_ != "classification":
            raise AttributeError("predict_proba solo está disponible para clasificación.")

        X = np.asarray(X)
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        proba_sum = np.zeros((n_samples, n_classes), dtype=float)

        for est in self.estimators_:
            # Probabilidades del árbol actual 
            est_proba = est.predict_proba(X)  
            est_classes = est.classes_         
            # Mapear columnas a índice global de clases
            # Dado que LabelEncoder ordena y las clases_ del árbol también están ordenadas,
            # podemos ubicar con searchsorted.
            idx_map = np.searchsorted(self.classes_, est_classes)
            # Sumar en las columnas correspondientes
            proba_sum[:, idx_map] += est_proba

        # Promedio
        proba_avg = proba_sum / float(len(self.estimators_))

        # Normalización por si algún árbol omitió clases
        row_sums = proba_avg.sum(axis=1, keepdims=True)
        # Evitar división por cero: si alguna fila suma 0, asignar uniforme
        zero_rows = (row_sums[:, 0] == 0.0)
        if np.any(zero_rows):
            proba_avg[zero_rows, :] = 1.0 / n_classes
            row_sums = proba_avg.sum(axis=1, keepdims=True)
        proba_avg /= row_sums

        return proba_avg

    # Compatibilidad mínima con API sklearn
    def get_params(self, deep: bool = True):
        return {
            "n_estimators": self.n_estimators,
            "max_features": self.max_features,
            "criterion": self.criterion,
            "max_depth": self.max_depth,
            "random_state": self.random_state,
            "task": self.task,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

### Elemento 1 - Preguntas teóricas

#### ¿Por qué el bagging ayuda a reducir la varianza del modelo?

El *bagging* reduce la varianza mediante la *agregación de predicciones de múltiples modelos entrenados independientemente*. Los árboles de decisión individuales, especialmente si son profundos, son modelos de *alta varianza*; y tienden a sobreajustarse al ruido de su conjunto de entrenamiento.

Al entrenar cada árbol ($h_t$) en una muestra *bootstrap* diferente, cada árbol aprende patrones ligeramente distintos y comete errores diferentes. Al final, el *bagging* combina sus predicciones ya sea por voto mayoritario en clasificación o promedio en regresión.

Este proceso de promediar las predicciones de muchos modelos decorrelacionados provoca que *los errores aleatorios individuales se anulen*. El resultado es un modelo de ensamble final que mantiene el bajo sesgo de los árboles individuales, pero con una varianza mucho menor.

#### ¿Qué efecto tiene limitar el número de variables consideradas en cada división?

Este mecanismo es la contribución clave que distingue a un *Random Forest* de un ensamble de *bagging* simple, ya que su efecto principal es *decorrelacionar los árboles del ensamble*.

Si no se limitaran las variables, todos los árboles tenderían a elegir la misma característica "más fuerte" en la primera división, y luego la siguiente mejor, y así sucesivamente. Esto haría que todos los árboles fueran estructuralmente muy similares, y por tanto, altamente correlacionados.

Al forzar a cada nodo a elegir la mejor división solo dentro de un *subconjunto aleatorio de características* (p. ej., `max_features='sqrt'`), se incrementa la diversidad estructural del bosque. Árboles más diversos producen un ensamble más robusto y preciso cuando sus votos se combinan.

#### ¿Cómo cambia el desempeño al incrementar el número de árboles en el ensamble?

Generalmente, al incrementar el número de árboles (`n_estimators`), el desempeño del modelo *mejora y converge*.

* Con pocos árboles, el ensamble es inestable y la varianza de la predicción es alta.
* A medida que se añaden más árboles, el voto/promedio se vuelve más robusto, el error de generalización converge y la varianza del ensamble disminuye.

A diferencia de otros métodos de ensamble, un Random Forest *no sufre de sobreajuste por añadir demasiados árboles*. El desempeño simplemente llega a un punto de equilibrio. El único costo de añadir más árboles más allá de ese punto es un mayor tiempo de entrenamiento y predicción.

## Elemento 2 - Comparativa con scikit-learn

In [ ]:
# Modelo alternativo usando RandomForest de sklearn directamente

from typing import Optional, Union
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.utils.multiclass import type_of_target


class SimpleRandomForestSK:
    """
    Envoltura ligera sobre RandomForest de scikit-learn con una interfaz simple y consistente.

    Parámetros
    ----------
    n_estimators : int, default=100
        Número de árboles.
    max_features : {'sqrt','log2','all'} o int o float en (0,1], default='sqrt'
        Subconjunto de características por división. 'all' -> usa todas (None en sklearn).
    criterion : str, default='gini' (clasificación) o 'squared_error' (regresión)
        Criterio del árbol base.
    max_depth : int o None, default=None
        Profundidad máxima de cada árbol.
    random_state : int o None, default=None
        Semilla de aleatoriedad.
    task : {'auto','classification','regression'}, default='auto'
        Tipo de problema; si 'auto', se infiere de y.

    Atributos (tras fit)
    --------------------
    estimator_ : RandomForestClassifier o RandomForestRegressor
        Modelo entrenado.
    task_ : str
        'classification' o 'regression'.
    classes_ : np.ndarray (solo clasificación)
        Clases del modelo.
    n_features_in_ : int
        Número de columnas de X usadas en el entrenamiento.
    """

    def __init__(
        self,
        n_estimators: int = 100,
        max_features: Union[str, int, float] = "sqrt",
        criterion: Optional[str] = None,
        max_depth: Optional[int] = None,
        random_state: Optional[int] = None,
        task: str = "auto",
    ):
        self.n_estimators = int(n_estimators)
        self.max_features = max_features
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.task = task

        self.estimator_ = None
        self.task_ = None
        self.classes_ = None
        self.n_features_in_ = None

    # Utilidades internas
    def _resolve_max_features(self, n_features: int):
        """Traduce 'all' a None y valida enteros/floats dentro de rango."""
        mf = self.max_features
        if mf == "all":
            return None
        if isinstance(mf, (int, np.integer)):
            if 1 <= mf <= n_features:
                return int(mf)
            raise ValueError("max_features int debe estar en [1, n_features].")
        if isinstance(mf, float):
            if 0.0 < mf <= 1.0:
                return float(mf)
            raise ValueError("max_features float debe estar en (0, 1].")
        if mf in (None, "sqrt", "log2"):
            return mf
        raise ValueError("max_features debe ser 'sqrt', 'log2', 'all', None, int o float en (0,1].")
    # Inferencia de tarea
    def _infer_task(self, y: np.ndarray) -> str:
        if self.task in ("classification", "regression"):
            return self.task
        y_type = type_of_target(y)
        if y_type in ("binary", "multiclass"):
            return "classification"
        elif y_type in ("continuous",):
            return "regression"
        # fallback heurístico
        return "classification" if np.issubdtype(np.asarray(y).dtype, np.integer) else "regression"

    # API principal
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X)
        y = np.asarray(y)

        if X.ndim != 2:
            raise ValueError("X debe ser 2D (n_samples, n_features).")
        if X.shape[0] != y.shape[0]:
            raise ValueError("X y y deben tener el mismo número de filas.")

        self.n_features_in_ = X.shape[1]
        self.task_ = self._infer_task(y)
        max_features = self._resolve_max_features(self.n_features_in_)

        if self.task_ == "classification":
            criterion = self.criterion if self.criterion is not None else "gini"
            self.estimator_ = RandomForestClassifier(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                max_features=max_features,
                criterion=criterion,
                bootstrap=True,
                random_state=self.random_state,
                n_jobs=-1,
            )
        else:
            criterion = self.criterion if self.criterion is not None else "squared_error"
            self.estimator_ = RandomForestRegressor(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                max_features=max_features,
                criterion=criterion,
                bootstrap=True,
                random_state=self.random_state,
                n_jobs=-1,
            )

        self.estimator_.fit(X, y)

        if self.task_ == "classification":
            self.classes_ = self.estimator_.classes_

        return self
    # Predicción
    def predict(self, X: np.ndarray) -> np.ndarray:
        if self.estimator_ is None:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        X = np.asarray(X)
        return self.estimator_.predict(X)
    # Probabilidades para clasificación
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        if self.estimator_ is None:
            raise RuntimeError("El modelo no está entrenado. Llama a fit(X, y) primero.")
        if self.task_ != "classification":
            raise AttributeError("predict_proba solo está disponible para clasificación.")
        X = np.asarray(X)
        return self.estimator_.predict_proba(X)

    # Compatibilidad mínima con API sklearn
    def get_params(self, deep: bool = True):
        return {
            "n_estimators": self.n_estimators,
            "max_features": self.max_features,
            "criterion": self.criterion,
            "max_depth": self.max_depth,
            "random_state": self.random_state,
            "task": self.task,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

### === SimpleRandomForest (propio) en Iris (clean) ===

**Accuracy:** 0.889

**Clases (orden interno):** `[0 1 2]`

**Matriz de confusión:**  
[[12 0 0]  
[ 0 10 2]  
[ 0 2 10]]  

**Reporte de clasificación:**

| | precision | recall | f1-score | support |
|:--- |:--- |:--- |:--- |:--- |
| **0** | 1.000 | 1.000 | 1.000 | 12 |
| **1** | 0.833 | 0.833 | 0.833 | 12 |
| **2** | 0.833 | 0.833 | 0.833 | 12 |
| | | | | |
| **accuracy** | | | 0.889 | 36 |
| **macro avg** | 0.889 | 0.889 | 0.889 | 36 |
| **weighted avg**| 0.889 | 0.889 | 0.889 | 36 |


### === RandomForest (sklearn wrapper) en Iris (clean) ===

**Accuracy:** 0.917

**Clases (orden interno):** `[0 1 2]`

**Matriz de confusión:**
[[12 0 0]  
[ 0 10 2]  
[ 0 1 11]]

**Reporte de clasificación:**

| | precision | recall | f1-score | support |
|:--- |:--- |:--- |:--- |:--- |
| **0** | 1.000 | 1.000 | 1.000 | 12 |
| **1** | 0.909 | 0.833 | 0.870 | 12 |
| **2** | 0.846 | 0.917 | 0.880 | 12 |
| | | | | |
| **accuracy** | | | 0.917 | 36 |
| **macro avg** | 0.918 | 0.917 | 0.917 | 36 |
| **weighted avg**| 0.918 | 0.917 | 0.917 | 36 |

### Elemento 2 - Preguntas teóricas

#### ¿Qué diferencias cuantitativas y cualitativas se observan entre tu implementación y la de sklearn?

* **Diferencia Cuantitativa:** La implementación de `scikit-learn` obtuvo un desempeño superior en el conjunto de prueba. `sklearn` alcanzó un **Accuracy de 0.917**, mientras que nuestra implementación `SimpleRandomForest` logró un **Accuracy de 0.889**.  

* **Diferencia Cualitativa:** Nuestra implementación `SimpleRandomForest` captura correctamente la lógica fundamental del *bagging*: un *loop* en Python que crea una muestra *bootstrap* (`_bootstrap_sample`) y entrena un `DecisionTreeClassifier` base en cada iteración. La versión de `sklearn`, por otro lado, es una implementación nativa altamente optimizada que gestiona la paralelización (reflejado en el uso de `n_jobs=-1`), la asignación de memoria y la aleatoriedad de forma mucho más eficiente.

#### ¿Cómo influyen los parámetros `n_estimators` y `max_features` en el desempeño del modelo?

Ambos hiperparámetros son cruciales debido a que controlan el balance sesgo-varianza del ensamble:

* `n_estimators` (Número de árboles): Controla la **robustez y estabilidad del ensamble**. Un valor mayor (ej. 100, 200) reduce la varianza del ensamble y estabiliza la predicción, hasta que el error converge.  

* `max_features` (Variables por división): Controla la **diversidad del ensamble**. Un valor pequeño por ejemplo ```sqrt``` produce árboles muy diferentes entre sí, lo que produce baja correlación, lo cual es ideal para reducir la varianza del ensamble. Un valor grande por ejemplo ```all, None```hace que los árboles sean muy correlacionados, pero individualmente con menos sesgo. El valor óptimo es un balance entre ambos.

#### ¿Por qué el modelo de `sklearn` suele ser más rápido o más preciso?

* **Rapidez:** Principalmente por la **paralelización** y la **optimización de bajo nivel**. El *wrapper* `SimpleRandomForestSK` utiliza `n_jobs=-1`, lo que permite a `sklearn` entrenar los múltiples árboles del bosque en paralelo. La implementación `SimpleRandomForest` entrena cada árbol secuencialmente dentro de un *loop* de Python. Además, el código de `sklearn` está optimizado en Cython, siendo mucho más rápido que instanciar y entrenar repetidamente objetos de Python.  

* **Precisión:** La ligera ventaja en precisión (0.917 vs 0.889) puede deberse a múltiples optimizaciones internas: heurísticas de división más avanzadas, un manejo diferente de la aleatoriedad para la selección de características, o diferencias sutiles en cómo se agregan las predicciones, por ejemplo nuestra implementación `SimpleRandomForest` usa `predict_proba` y `argmax` para `predict`, mientras que `sklearn` implementa un voto mayoritario directo más optimizado, entre otras posibles causas.

#### ¿Tu implementación mantiene el mismo comportamiento al modificar la semilla aleatoria?

No. Si se modifica el `random_state`, nuestra implementación (`SimpleRandomForest`) generará un ensamble completamente diferente.

La semilla (`random_state`) controla dos procesos aleatorios críticos en nuestro código:
1.  La generación de las muestras *bootstrap* (a través de `_bootstrap_sample`).
2.  La semilla individual que se pasa a cada `DecisionTreeClassifier`, que a su vez controla la aleatoriedad en la selección de `max_features` en cada división.

Cambiar la semilla resultará en un modelo distinto, y es esperable que el *accuracy* varíe ligeramente.

## Elemento 3 - Creación y despliegue de la API

Para realizar el despliegue no se necesitó escribir código ya que render se encarga de eso, sin embargo para la creación de la API se debe definir un módulo main.py que orqueste las peticiones, dentro de ese modulo se coloca el siguiente código que crea el objeto de la API, define y maneja los endpoints.

Este código por si solo no hace nada si se ejecuta en este notebook, pero es indispensable al momento de desplegar la API.

Adicionalmente para realizar las predicciones con nuestro modelo se definió un modulo srf_model.py que contiene la clase del modelo personalizado.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, conlist
import os
import joblib
import numpy as np
import sys
import app.srf_model as srf_model # Importa el módulo del modelo personalizado

app = FastAPI(title="Random Forest API", version="1.0.0")
MODEL_PATH = os.getenv("MODEL_PATH", "model/srf_propio_model.pkl")

try:
    sys.modules['__main__'] = srf_model
    model = joblib.load(MODEL_PATH)
except Exception as e:
    raise RuntimeError(f"No se pudo cargar el modelo desde {MODEL_PATH}: {e}")

# Esquemas de entrada/salida
class PredictRequest(BaseModel):
    features: conlist(float, min_length=1) # type: ignore

class PredictResponse(BaseModel):
    prediction: str


@app.get("/health")
def health():
    return {"status": "ok"}

@app.get("/info")
def info():
    return {
        "team": "GPT-4o mini",
        "model": type(model).__name__,
        "n_estimators": getattr(model, "n_estimators", None),
        "max_depth": getattr(model, "max_depth", None),
    }

@app.post("/predict", response_model=PredictResponse)
def predict(req: PredictRequest):
    try:
        X = np.array(req.features, dtype=float).reshape(1, -1)
        pred = model.predict(X)[0]
        iris_map = {0: "setosa", 1: "versicolor", 2: "virginica"}
        if isinstance(pred, (int, np.integer)):
            pred = iris_map.get(pred, str(pred))
        return {"prediction": str(pred)}
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Solicitud inválida: {e}")


### Elemento 3 - Preguntas teóricas

#### ¿Qué ventajas ofrece exponer un modelo como servicio web?

Exponer un modelo como un servicio web (API) es la práctica estándar en un entorno de producción. La ventaja principal es el **desacoplamiento**:

* **Accesibilidad Universal:** Cualquier aplicación de tipo móvil, web, otro *backend*, sin importar el lenguaje en que esté escrita, puede consumir el modelo simplemente realizando una petición HTTP.  

* **Mantenimiento Centralizado:** Se puede actualizar, reentrenar y desplegar una nueva versión del modelo en el servidor de API (actualizando el `model.pkl`) sin necesidad de actualizar o redistribuir las aplicaciones cliente que lo consumen.  

* **Escalabilidad Independiente:** El servicio del modelo puede escalarse independientemente de la aplicación principal, permitiendo gestionar picos de demanda de inferencia de forma eficiente.

#### ¿Qué riesgos o limitaciones pueden surgir si no se valida correctamente la entrada del usuario?

La validación de entrada es una medida de seguridad y robustez fundamental. Si no se valida, los riesgos pueden ser:

* **Fallas del Servicio o Crashes:** Si el modelo espera 4 *features* y la API recibe 3, o recibe texto en lugar de números, el *script* de predicción fallará (ej. `ValueError`). Esto causará un error 500 y potencialmente interrumpirá el servicio para todos los usuarios.  

* **Predicciones Erróneas o Silenciosas:** El modelo puede recibir datos en un orden incorrecto, o con valores físicamente imposibles, por ejemplo una longitud de pétalo de -100. El modelo podría no fallar, pero devolvería una predicción absurda.  

* **Riesgos de Seguridad:** En casos más complejos, entradas malformadas podrían explotar vulnerabilidades de *deserialización* en caso de usar `pickle` de forma insegura o de inyección de código.

#### ¿Por qué es importante incluir un endpoint de `/health` en una API?

El *endpoint* `/health` es una herramienta muy importante de **monitoreo y orquestación**.

Permite a sistemas automatizados verificar constantemente que el servicio no solo está *corriendo*, sino que está *saludable*, es decir, responde activamente a peticiones y está listo para operar.

Si el *health check* falla, el orquestador puede automáticamente reiniciar el servicio o dejar de enviarle tráfico, para garantizar la fiabilidad y la disponibilidad del sistema para los usuarios finales.

#### ¿Cómo podrías garantizar que tu servicio mantenga disponibilidad bajo diferentes condiciones?

Para garantizar la "alta disponibilidad" podríamos aplicar las siguientes acciones:

1.  **Redundancia y Balanceo de Carga:** Desplegar múltiples instancias idénticas del servicio API detrás de un *balanceador de carga*. Si una instancia falla, el balanceador redirige el tráfico a las instancias saludables.

2.  **Manejo de Carga Concurrente:** Utilizar un servidor web asíncrono que pueda manejar muchas peticiones simultáneas eficientemente.

3.  **Validación Robusta de Entradas:** Asegurar que la API maneje las peticiones malformadas sin fallar o "crasheando".

4.  **Monitoreo y Alertas:** Usar el *endpoint* `/health` para monitorear activamente el servicio y alertar en caso de que falle.

5.  **Manejo de Recursos:** Asegurar que el servicio tenga suficiente memoria y CPU, y que no entre en modo "sleep" inesperadamente, en caso de estar utilizando la versión gratuita de Render.